<a href="https://colab.research.google.com/github/Anro607/word2vec_in_literature/blob/main/doribasil_will_WIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2023학년도 교과융합 탐구 보고서 - 고전문학에 워투벡을 얹어드셔보세요(가제)
https://www.notion.so/ae8770f8f03e4028a746618d5074831c
<-내 노션바로가기

In [68]:
#웹 크롤
import requests

#전처리
import re
import zipfile
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')


#워드투벡
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 1. 웹 크롤링, 문자열 인덱싱으로 전처리하기

In [ ]:
#웹 크롤링을 사용해 gutenberq에서 업로드한 도리온 그레이의 초상 원문 가져오기

url = "https://www.gutenberg.org/cache/epub/174/pg174.txt"  #Plain Text UTF-8
all_html = requests.get(url)                                #가져오기
print(all_html.text)                                        #제대로 가져와졌는지 확인

In [ ]:
dorion = all_html.text                                      #전체 내용 텍스트

현재 dorion 변수에는 도리온 그레이의 초상 원문 뿐 아닌 구텐베르크 각주, 오스카와일드 작가의 말, 목차 등 분석에 불필요한 요소가 포함됨.

남겨야 할 것:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a. 본문

삭제해야 할 것:
1. 'The Project Gutenberg eBook of The Picture of Dorian Gray' ~ 'CHAPTER I.'
2. 'CHAPTER II', 'CHAPTER III' 등 I~XX 챕터 문구.
3. '  *** END OF THE PROJECT GUTENBERG EBOOK THE PICTURE OF DORIAN GRAY' ~ 끝까지

이를 a만 남기고, 본문 사이 2를 없애는 과정으로 어쩌구

In [ ]:
one = dorion.find('The studio was filled with the rich odour of roses,')                    #첫문장 (3314)
three = dorion.find('THE END')                                                              #디엔드 (438613)
print(one)
print(three)

3314
438613


고로 dorion 문자열 변수에서 0\~3314 인덱스와 438613\~끝 인덱스를 잘라야 하기에
반대로 3314\~438612 인덱스를 남길 것이다.

(438613이 아닌 438612을 잘라야 THE END의 'T'가 사라짐)

In [ ]:
dorion = dorion[3314: 438612]
print(dorion)

챕터 표기를 포함한 본문을 뽑아냄!

In [75]:
chapter = ['CHAPTER II.','CHAPTER III.','CHAPTER IV.','CHAPTER V.','CHAPTER VI.'
          ,'CHAPTER VII.','CHAPTER VIII.','CHAPTER IX.','CHAPTER X.','CHAPTER XI.'
          ,'CHAPTER XII.','CHAPTER XIII.','CHAPTER XIV.','CHAPTER XV.','CHAPTER XVI.',
           'CHAPTER XVII.','CHAPTER XVIII.','CHAPTER XIX.','CHAPTER XX.']                 #챕터리스트리스트 엌ㅋㅋ
dorion = a                                                                                     #원본 보호를 위한 임시 변수
for i in chapter:
  a = a.replace(i,'')                                                                     #제거

print(a.find('CHAPTER'))                                                                  #잘 제거됐는지 확인(-1은 해당 문자가 없다는 뜻!)

-1


필요한 요소만 남기는 일에 성공했으니 이제 진짜 토큰화를 해보자.

참고한 글 = https://wikidocs.net/21698

# 2. 토큰화

In [71]:
token_first = sent_tokenize(a)                                                     #토큰화 준비

normalized_text = []                                                               #변환한 문장을 넣을 리스트
for string in token_first:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())                           #각 문장의 구두점을 제거하고, 대문자를 소문자로 변환
     normalized_text.append(tokens)


result = [word_tokenize(sentence) for sentence in normalized_text]                 #토큰화 수행

print('총 샘플의 개수 : {}'.format(len(result)))
for line in result[:3]:
    print(line)



총 샘플의 개수 : 5373
['the', 'studio', 'was', 'filled', 'with', 'the', 'rich', 'odour', 'of', 'roses', 'and', 'when', 'the', 'light', 'summer', 'wind', 'stirred', 'amidst', 'the', 'trees', 'of', 'the', 'garden', 'there', 'came', 'through', 'the', 'open', 'door', 'the', 'heavy', 'scent', 'of', 'the', 'lilac', 'or', 'the', 'more', 'delicate', 'perfume', 'of', 'the', 'pink', 'flowering', 'thorn']
['from', 'the', 'corner', 'of', 'the', 'divan', 'of', 'persian', 'saddle', 'bags', 'on', 'which', 'he', 'was', 'lying', 'smoking', 'as', 'was', 'his', 'custom', 'innumerable', 'cigarettes', 'lord', 'henry', 'wotton', 'could', 'just', 'catch', 'the', 'gleam', 'of', 'the', 'honey', 'sweet', 'and', 'honey', 'coloured', 'blossoms', 'of', 'a', 'laburnum', 'whose', 'tremulous', 'branches', 'seemed', 'hardly', 'able', 'to', 'bear', 'the', 'burden', 'of', 'a', 'beauty', 'so', 'flamelike', 'as', 'theirs', 'and', 'now', 'and', 'then', 'the', 'fantastic', 'shadows', 'of', 'birds', 'in', 'flight', 'flitted', 'acr

불용어 제거!! https://wikidocs.net/22530 참고



In [118]:
temp_result = result                                                                #원본 손실 방지
stop_words = set(stopwords.words('english'))                                        #nltk에서 정한 영어 불용어
#stop_words.update(['one', 'would'])                                #이후 테스트에서 확인한 제거되지 않은 불용어

end_result = []                                                                     #불용어가 아닌 단어만 넣을 리스트
for list_ in temp_result:
  for word in list_ :
    if word in stop_words:
        list_.remove(word)
  end_result.append(list_)

print('총 샘플의 개수 : {}'.format(len(result)))
for line in end_result[:3]:
    print(line)

총 샘플의 개수 : 5373
['studio', 'filled', 'rich', 'odour', 'roses', 'light', 'summer', 'wind', 'stirred', 'amidst', 'trees', 'garden', 'came', 'open', 'door', 'heavy', 'scent', 'lilac', 'delicate', 'perfume', 'pink', 'flowering', 'thorn']
['corner', 'divan', 'persian', 'saddle', 'bags', 'lying', 'smoking', 'custom', 'innumerable', 'cigarettes', 'lord', 'henry', 'wotton', 'could', 'catch', 'gleam', 'honey', 'sweet', 'honey', 'coloured', 'blossoms', 'laburnum', 'whose', 'tremulous', 'branches', 'seemed', 'hardly', 'able', 'bear', 'burden', 'beauty', 'flamelike', 'fantastic', 'shadows', 'birds', 'flight', 'flitted', 'across', 'long', 'tussore', 'silk', 'curtains', 'stretched', 'front', 'huge', 'window', 'producing', 'kind', 'momentary', 'japanese', 'effect', 'making', 'think', 'pallid', 'jade', 'faced', 'painters', 'tokyo', 'medium', 'art', 'necessarily', 'immobile', 'seek', 'convey', 'sense', 'swiftness', 'motion']
['sullen', 'murmur', 'bees', 'shouldering', 'way', 'long', 'unmown', 'grass', 

# 3. 워드 투 벡터 학습

In [132]:
model = Word2Vec(sentences=end_result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [138]:
model_result = model.wv.most_similar("man")
print(model_result)

[('things', 0.9997504949569702), ('round', 0.9997105598449707), ('life', 0.9996974468231201), ('away', 0.9996891617774963), ('back', 0.999683678150177), ('time', 0.9996836185455322), ('men', 0.9996787905693054), ('old', 0.9996744990348816), ('face', 0.9996742010116577), ('long', 0.9996737837791443)]


In [102]:
a = input("찾고싶은 특정 단어의 유사단어 목록 : ")
model_result = model.wv.most_similar(a)
print(model_result)

찾고싶은 특정 단어의 유사단어 목록 : face
[('room', 0.9997405409812927), ('one', 0.9997245669364929), ('would', 0.9997214078903198), ('life', 0.9997110962867737), ('never', 0.9996740221977234), ('back', 0.9996738433837891), ('long', 0.9996717572212219), ('white', 0.9996669888496399), ('moment', 0.9996634125709534), ('away', 0.9996597766876221)]


밑에는 놀면서 저장한거..

In [140]:
model.wv.save_word2vec_format('doribasil') # 모델 저장 요건 픽쳐에 도리바질 다들가있더라ㄷㄷ

In [142]:
loaded_model = KeyedVectors.load_word2vec_format("doribasil") # 모델 로드...인데모델파일어딨는거임?!!!?!?!?!?